In [ ]:
import keras
keras.__version__

'2.5.0'

# CNN

05/2020

# Resumo

Neste notebook discutiremos o que é uma Rede Neural Convolucional, construiremos uma baseados nos conceitos aprendidos, e a utilizaremos para resolver um problema de classificação binária.

# Sumário

 <ol>
<li>1.Introdução</li>
<li>2.Detalhes da Arquitetura</li>
<li>3.Problema: Classificação de Imagens</li>
<li>4.Bibliografia</li>
</ol>

## 1.Introdução

Apesar de todas as dificuldades provenientes de se treinar Redes Neurais com muitas camadas intermediárias, ao longo dos anos foram desenvolvidas algumas técnicas para que se pudesse treinar esse tipo de Rede. Na área de reconhecimento e processamento de imagens, o tipo de rede neural que tem dominado as recentes pesquisas é a Rede Neural Convolucional (ou CNN da sigla em inglês) cuja arquitetura é desenvolvida de modo que seja possível treinar as redes de maneira eficiente.

## 2.Detalhes da Arquitetura


### 2.1.Local Receptive Fields

Diferente das Redes Neurais comuns, em que as camadas eram interpretadas como uma linha vertical de neurônios, nas Redes Neurais Convolucionais, as camadas de entrada devem ser interpretadas como um quadrado ou retângulo de neurônios como na imagem a seguir:


![alt text](http://neuralnetworksanddeeplearning.com/images/tikz42.png)

No problema do MNIST, por exemplo, todas as imagens estão em escala de cinza em dimensão 28x28, portanto a camada de entrada de uma CNN que resolva esse problema deve ser quadrada de dimensão também 28x28.

Contudo, nem todo conjunto de imagens é tão uniforme, é comum que as imagens estejam em dimensões diferentes umas das outras. Por isso é uma prática comum re-escalar as imagens no pré-processamento de modo que todas fiquem com dimensões iguais às da camada de entrada da CNN.

Diferente das camadas densas, onde todos os neurônios de uma camada são conectados com todos os neurônios da próxima camada, nas redes neurais convolucionais, cada neurônio da primeira camada intermediária conecta um número limitado de neurônios da camada de entrada. Para exemplificar será utilizada uma ilustração:

![alt text](http://neuralnetworksanddeeplearning.com/images/tikz44.png)

Podemos ver que um neurônio está conectado a um quadrado 5x5 de neurônios na entrada, essa pequena região que é apenas um pedaço da imagem original é chamado de *Campo Receptivo Local*. O neurônio da um peso para conexão que faz na entrada para aprendizagem, e ainda tem um bias. É possível interpretar isso como se o neurônio estivesse "aprendendo a analisar apenas seu próprio Campo Receptivo".

Para completar a próxima camada da CNN é como se os Campos Receptivos Locais "deslizassem" pela imagem até completá-la, gerando assim um neurônio na primeira camada intermediária para cada Campo Receptivo, seguindo a próxima imagem:

![alt text](http://neuralnetworksanddeeplearning.com/images/tikz45.png)

Seguindo assim até que todos os Campos Receptivos da imagem sejam contemplados, podemos ver que para 28x28 neurônios de entrada teremos apenas 24x24 na próxima camada, isso se dá pelo fato de utilizarmos um tamanho para os Campos Receptivos de 5x5.

### 2.2.Feature Maps

Um detalhe crucial para a arquitetura de uma CNN é que, por mais que cada neurônio de uma camada intermediária esteja conectado a apenas um pequeno pedaço da imagem, todos eles compartilham os mesmos pesos e bias. Sendo assim cada um dos 24x24 neurônios do exemplo que estamos construindo treinarão exatamente os mesmos pesos e bias. A saída de um neurônio qualquer dessa camada pode ser escrita por:

$$ \sigma \left(b + \sum \limits _{l = 0} ^{4} \sum \limits _{m = 0} ^{4} w_{l, m} a_{j+l,k+m} \right) (1)$$ 

Aqui seguindo a linguagem do Nielsen<sup>[1]</sup>, $\sigma$ é a função de ativação da rede neural, $b$ é o valor do bias e $w_{l, m}$ é conjunto 5x5 de pesos, todos compartilhados. Além disso, é utilizado $a_{x,y}$ para denotar a ativação do neurônio da posição $(x,y)$ da camada de entrada.

Esse método de compartilhamento de pesos e bias leva a um atributo interessante, que é que todos os neurônios desta camada estão aprendendo a analisar o mesmo recurso (ou feature) apenas em locais diferentes da imagem.

Por esse motivo, o mapeamento dos dados da entrada para a camada intermediária comummente recebe o nome de *Mapa de Recursos* (ou feature map). Como cada feature map só faz a detecção de um recurso da imagem, para fazer um reconhecimento eficiente de imagens, precisamos de mais de um feature map.


![alt text](http://neuralnetworksanddeeplearning.com/images/tikz46.png)

O exemplo acima tem 3 mapas de recursos, cada qual com 5x5 pesos compartilhados e 1 bias. Isso significa que essa Rede pode detectar 3 diferentes tipos de features, cada uma podendo ser identificada ao longo de toda imagem. Apesar da imagem mostrar apenas 3 features, frequentemente se utiliza muito mais feature maps em uma só camada da Rede Neural.

### 2.3.Pooling Layers

Outro elemento presente na arquitetura das Redes Neurais Convolucionais são  as chamadas *Camadas de Agrupamento* (ou Pooling Layer), essas camadas habitualmente estão presentes depois de uma camada de Mapa de Recursos, e seu papel é simplificar a informação presente na Camada Convolucional.

Cada Camada de Agrupamento pega a saída de um determinado Feature Map e gera uma versão condensada do mesmo, como se sintetizasse suas informações em um Feature Map menor. Por exemplo, cada unidade da Pooling Layer pode sintetizar uma região de 2x2 ou 3x3 da camada anterior, transformando toda essa informação em apenas um neurônio.

Existem várias maneiras de fazer essa síntese de informação, mas a maneira mais comum, que também sera utilizada neste notebook, é a chamada de Max-Pooling. Ela pega a região a ser condensada, e tem como saída o maior valor de ativação da mesma.

Como existem vários Feature Maps em uma só CNN, se aplica uma Pooling Layer para cada Feature Map, como na imagem a seguir:

![alt text](http://neuralnetworksanddeeplearning.com/images/tikz48.png)

### 2.4.Arquitetura Final

Agora que sabemos todos os artefatos de uma Rede Neural Convolucional, podemos juntar todas as partes e verificar como é a estrutura de uma CNN completa. Para isso podemos simplesmente ligar todas as saídas da Max Pooling Layer e conectar densamente à camada de saída da nossa Rede Neural da seguinte forma:

![alt text](http://neuralnetworksanddeeplearning.com/images/tikz49.png)

A camada de saída da Rede Neural é idêntica às Redes Neurais rasas, pode ter ativação do tipo SOFTMAX para classificação por exemplo, ou sigmóide caso o problema seja de classificação binária.

É comum ainda fazer adições a essa estrutura que criamos, como por exemplo adicionar mais Camadas Convolucionais seguidas de Pooling Layers, ou adicionar uma Camada Densa de Neurônios (completamente conectada), antes da camada de saída. Não há regra para essas alterações e elas devem se adequar ao problema atacado.











## 3.Problema: Classificação de Imagens

Para exemplificar a utilização das CNNs nesse notebook, resolveremos um problema que seria difícil de lidar de outra maneira. Esse problema é o de classificação de imagens de Cachorros e Gatos. O objetivo é aparentemente simples, devemos classificar imagens de cachorros e gatos, para isso utilizaremos o dataset "Dogs_Vs_Cats" presente no Kaggle, com mais de 25 mil imagens que são parecidas com as seguintes:

![alt text](https://camo.githubusercontent.com/aa544d57e3ba791348677ce6b8287394c85f76ba/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f626f6f6b2e6b657261732e696f2f696d672f6368352f636174735f76735f646f67735f73616d706c65732e6a7067)


Vamos treinar uma Rede Neural Convolucional para que ela seja capaz de, quando receber uma imagem, classificar se nela está presente um cachorro, ou um gato. Será utilizada apenas uma parte do dataset, com 20 mil imagens de treino e 2 mil para validação.

O primeiro passo é fazer a instalação do kaggle:

In [ ]:
!pip install kaggle

Depois devemos fazer o upload do API Token (obtida no site do kaggle), para o computador remoto, para isso podemos utilizar esse código:

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes


Então podemos finalmente baixar o dataset direto do Kaggle, e descomprimí-lo com:




In [ ]:
!mkdir $HOME/.kaggle
!mv kaggle.json $HOME/.kaggle/kaggle.json
!chmod 600 $HOME/.kaggle/kaggle.json


! kaggle competitions download -c 'dogs-vs-cats'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 33.3MB/s]
 96% 520M/543M [00:03<00:00, 157MB/s]
100% 543M/543M [00:03<00:00, 154MB/s]
 94% 254M/271M [00:01<00:00, 237MB/s]
100% 271M/271M [00:01<00:00, 232MB/s]


In [ ]:
!unzip -qq /content/train.zip -d /datalab/ 

Agora, com todas as imagens no computador remoto, vamos separá-las em pastas de modo que exista uma pasta para treino, uma para validação e uma para as imagens de teste.

Nós pegaremos as 10 mil primeiras imagens de cachorros e de gatos e moveremos para a pasta de treino, e colocaremos mil imagens de cada tipo nas pastas de validação e teste. Em cada uma dessas pastas, cachorros e gatos serão separados em pastas diferentes.


Podemos fazer isso com o código a seguir:

In [ ]:
import os, shutil

original_dataset_dir = '/datalab/train/'


dir_base = '/content/processed_datalab'
if not os.path.exists(dir_base):
    os.mkdir(dir_base)

dir_treino = os.path.join(dir_base, 'train')
if not os.path.exists(dir_treino):
    os.mkdir(dir_treino)
dir_valid = os.path.join(dir_base, 'validation')
if not os.path.exists(dir_valid):
    os.mkdir(dir_valid)
dir_teste = os.path.join(dir_base, 'test')
if not os.path.exists(dir_teste):
    os.mkdir(dir_teste)

train_cats_dir = os.path.join(dir_treino, 'cats')
if not os.path.exists(train_cats_dir):
    os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(dir_treino, 'dogs')
if not os.path.exists(train_dogs_dir):
    os.mkdir(train_dogs_dir)

validation_cats_dir = os.path.join(dir_valid, 'cats')
if not os.path.exists(validation_cats_dir):
    os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(dir_valid, 'dogs')
if not os.path.exists(validation_dogs_dir):
    os.mkdir(validation_dogs_dir)

test_cats_dir = os.path.join(dir_teste, 'cats')
if not os.path.exists(test_cats_dir):
    os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(dir_teste, 'dogs')
if not os.path.exists(test_dogs_dir):
    os.mkdir(test_dogs_dir)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(10000, 11000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(11000, 12000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(10000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(10000, 11000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(11000, 12000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

Mesmo com as imagens devidamente separadas, elas ainda não estão prontas para entrar em uma CNN. É necessário que se faça um pré-processamento das imagens para que todas fiquem na mesma escala e dimensão para que possam ser reconhecidas pela camada de entrada da nossa Rede Neural Convolucional. Mudaremos todas as imagens para escala de cinza de 8 bits, e deixaremos todas com medida 150x150.

Utilizaremos o *ImageDataGenerator* do Keras.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        dir_treino,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        dir_valid,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Com nossas imagens prontas, vamos desenvolver agora a nossa Rede Neural.

Vamos adicionar 4 conjuntos Convolutional Layer e Max Pooling, com entrada 150x150, todas com Campos Receptivos Locais 5x5 e Região de Agrupamento 2x2. O número de feature maps irá aumentando ao longo das camadas, começando com 20, passando por 40 e tendo duas Camadas Convolucionais de 80 feature maps. A função de ativação utilizada será o Retificador Linear (ReLU).

Podemos criar essa Rede Neural com o seguinte código:

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(16, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

Como as camadas até aqui tem duas dimensões, e as próximas camadas utilizadas tem apenas uma dimensão, precisaremos "achatar" os dados, de modo que eles fiquem com apenas uma dimensão, podemos fazer isso dessa maneira:

In [ ]:
model.add(layers.Flatten())

Agora adicionaremos uma camada com regularização do tipo dropout, para diminuir o efeito do overfitting, e adicionaremos uma camada densa de 100 neurônios, além da saída com a função de ativação sigmóide.

In [ ]:
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Os hiper-parâmetros escolhidos neste notebook foram escolhidos baseados nos notebooks do Chollet<sup>[2]</sup>, no livro do Nielsen<sup>[1]</sup> e em testes empíricos.

Com a função *summary* podemos ver como ficou nossa rede neural:

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9248)              0

Agora precisamos escolher nossos otimizadores:

In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


E enfim, podemos fazer o treino da nossa CNN, como os dados foram gerados pelo *DataGenerator*, nós teremos que utilizar a função fit_generator. Treinaremos para 50 épocas.

In [ ]:
print(train_generator)

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
100/100 [==============================] - 53s 96ms/step - loss: 0.7256 - acc: 0.5049 - val_loss: 0.6726 - val_acc: 0.6330
Epoch 2/100
100/100 [==============================] - 9s 86ms/step - loss: 0.6813 - acc: 0.5718 - val_loss: 0.6746 - val_acc: 0.5510
Epoch 3/100
100/100 [==============================] - 9s 91ms/step - loss: 0.6665 - acc: 0.5943 - val_loss: 0.6412 - val_acc: 0.6490
Epoch 4/100
100/100 [==============================] - 9s 91ms/step - loss: 0.6548 - acc: 0.6003 - val_loss: 0.6247 - val_acc: 0.6500
Epoch 5/100
100/100 [==============================] - 9s 88ms/step - loss: 0.6325 - acc: 0.6293 - val_loss: 0.6659 - val_acc: 0.5810
Epoch 6/100
100/100 [==============================] - 8s 84ms/step - loss: 0.6073 - acc: 0.6689 - val_loss: 0.6142 - val_acc: 0.6680
Epoch 7/100
100/100 [==============================] - 8s 85ms/step - loss: 0.5998 - acc: 0.6891 - val_loss: 0.5878 - val_acc: 0.6870
Epoch 8/100
100/100 [==============================] - 8s 84m

In [ ]:
model.save('cats_and_dogs_test.h5')

Agora podemos avaliar os resultados:

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('cats_and_dogs_small_2.h5')

In [ ]:
import numpy as np

np.shape(train_generator[0][0][0])

(150, 150, 3)

A instabilidade da função de custo mostra que ainda há espaço para melhoria, apesar disso, a CNN criada do zero conseguiu atingir ótimos resultados, uma acurácia de cerca de 87%.

##5. Bibliografia 

[1] http://neuralnetworksanddeeplearning.com/index.html

[2] https://github.com/fchollet/deep-learning-with-python-notebooks

[3] https://colab.research.google.com/drive/19SVdlmnn6yRXCvNnE8PT1vbXrA8FrBo_#scrollTo=1HrGpk_b4dvL

[4] https://www.kaggle.com/c/dogs-vs-cats

[5] https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/
